# Financial Data Preprocessing
A time series is a series of data points indexed in time order. Financial Data such as equity, commodity, and forex price series observed at equally spaced points in time are an example of such a series. It is a sequence of data points observed at regular time intervals and depending on the frequency of observations, a time series may typically be in ticks, seconds, minutes, hourly, daily, weekly, monthly, quarterly and annual.

The first step towards any data analysis would be to parse the raw data that involves extracting the data from the source and then cleaning and filling the missing data if any. While data comes in many forms, Python makes it easy to read time-series data using useful packages.

In this session, we will retrieve and store both end-of-day and intraday data using some of the popular python packages. These libraries aim to keep the API simple and make it easier to access historical data. Further, we will see how to read data from traditional data sources stored locally.

In [ ]:
# Ignore warnings - optional
import warnings
warnings.filterwarnings('ignore')

# Import data manipulation libraries
import numpy as np
import pandas as pd
        
# Import yahoo finance library
import yfinance as yf

# Data Retrieval
Retrieving EOD, Intraday, Options data

Retrieving end-of-day data for single security
We'll retrieve historical data from yahoo finance using yfinance library

In [ ]:
# Example 1
# Fetch the data by specifying the number of period
df1 = yf.download("AAPL", period="10d", progress=False)

# Display the first five rows of the dataframe to check the results. 
df1.head()

In [ ]:
# Example 2
# Fetch data by specifying the the start and end dates
df2 = yf.download("AAPL", start="2024-01-01", end="2024-01-31", progress=False)

# Display the first five rows of the dataframe to check the results. 
df2.head()

In [ ]:
# Display last five rows
df2.tail()

In [ ]:
# Example 3
# Fetch data for year to date (YTD)
df3 = yf.download("AAPL", period="ytd", progress=False)

# Display the last five rows of the dataframe to check the results. 
df3.tail()

## Retrieving data for multiple securities
We'll retrieve historical price data of five stocks from yahoo finance.



In [ ]:
# Specify stocks
# https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average
dow_stocks = ['UNH', 'GS', 'HD', 'AMGN', 'MCD']
dow_stocks

In [ ]:
# Fetch data for multiple stocks at once
df4 = yf.download(dow_stocks, period='ytd', progress=False)['Adj Close']

# Display dataframe
df4.tail()

### Retrieving multiple fields for multiple securities
We'll now retrieve multiple fields from yahoo finance.

In [ ]:
# Example 5
# Group the stocks
df5 = yf.download(dow_stocks, start="2024-06-01", end="2024-06-30", group_by="ticker", progress=False)

In [ ]:
df5.head()


In [ ]:
# Display GS stock data
df5['GS']

### Retrieving intraday data
We'll now retrieve intraday data from yahoo finance.

In [ ]:
# Example 6
# Retrieve intraday data for last five days
df6 = yf.download('AAPL', period='5d', interval='1m', progress=False)

# Display dataframe
df6

### Retrieving option chain
We'll now retrieve option chain for SPY for March 2024 expiration from yahoo finance and filter the output to display the first seven columns.

In [ ]:
# Ticker object
spy = yf.Ticker('SPY')

In [ ]:
spy.options

In [ ]:
# Get SPY option chain for March 28th expiration
# https://finance.yahoo.com/quote/SPY240930C00545000
options = spy.option_chain('2024-09-30')
options

In [ ]:
# Filter calls for strike above 545
df7 = options.calls[(options.calls['strike']>540) & (options.calls['strike']<550)]

# Check the filtered output
df7.iloc[:,:7]

Retrieving Hypertext Markup Language (HTML)
We'll now retrieve India's benchmark index NIFTY50 Index data from Wikipedia.

In [ ]:
pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[1]


In [ ]:
# Read data from wikipedia
nifty50 = pd.read_html('https://en.wikipedia.org/wiki/NIFTY_50')[2].Symbol.to_list()

# Read five symbols
nifty50[:5]

## Database Storage & Retrieval
Let's store the data in the database and load back for manipulation.


Storing Nifty50 data in SQLite Database

In [ ]:
# %load_ext autoreload
# %autoreload 1

# Import & create database 
# https://sqlite.org/cli.html
from sqlalchemy import create_engine, text
engine = create_engine('sqlite:///India')

In [ ]:
# Fetch data from yahoo using list comprehension
data = [yf.download(symbol+'.NS', period='250d', progress=False).reset_index() for symbol in nifty50] 

# save it to database
for frame, symbol in zip(data, nifty50):
    frame.to_sql(symbol, engine, if_exists='replace', index=False)

In [ ]:
# Reading Sqlite Database
# We'll now read the ohlcv data stored locally in the database using Pandas

# Query from database
query = 'SELECT * FROM  "RELIANCE" WHERE DATE > "2024-06-01"'

with engine.connect() as connection:
    result = pd.read_sql_query(text(query), connection)

# Display the results
result.tail()

Data Manipulation
Next, we'll see some methods used in data wrangling. This is critical if you work on financial data or time series.

Filter & Query Methods
We'll now see some examples of subset selection using Panda's filter and query methods.

In [ ]:
# Query from database
query = 'SELECT * FROM  "TITAN"'
with engine.connect() as connection:
    df = pd.read_sql_query(text(query), connection)

# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Set the date column as the DataFrame index
df.set_index('Date', inplace=True)

In [ ]:
# Get first few days of data
df.first('3D')

In [ ]:
# Get last few days of data
df.last('3D')

In [ ]:
# Filter based on column
df.filter(['Close'])

In [ ]:
# Filter based on row or index
df.filter(like="2024-06-24", axis=0)

In [ ]:
# Query for a specific condition. Ex: Close price > 3800
df.query('Close>3800')

In [ ]:
# Query for condition where the difference between High and Low is greater than 150
df.query('High-Low>150')

In [ ]:
# Query for a multiple conditions. Ex: Close > 3700 and High-Low > 150
df.query('Close>3700 & High-Low>150')

In [ ]:
# Query for condition where Close>Open price
df.query('Close>Open')

In [ ]:
# Query for condition where Open price is equal to Low price of the day
df.query('Open==Low')

In [ ]:
# Resampling data
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
# Resampling to derive weekly values from daily time series
df_weekly = df.resample('W').last()

# Display the last five rows of the data frame to check the output
df_weekly.tail()

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
# Resampling to derive weekly values from daily time series
df_weekly = df.resample('W').last()

# Display the last five rows of the data frame to check the output
df_weekly.tail()
# Resampling to a specific day of the week: Thu
df_weekly_thu = df.resample('W-THU').last()

# Display the last five rows of the data frame to check the output
df_weekly_thu.tail()

In [ ]:
# Resampling to derive monthly values from daily time series
df_monthly = df.resample('M').last()

# Display the last five rows of the data frame to check the output
df_monthly.tail()

## Interactive Visualization of Time Series
We use cufflinks for interactive visualization. It is one of the most feature rich third-party wrapper around Plotly by Santos Jorge. It binds the power of plotly with the flexibility of pandas for easy plotting.

When you import cufflinks library, all pandas data frames and series objects have a new method .iplot() attached to them which is similar to pandas' built-in .plot() method.

In [ ]:
# Import cufflinks for visualization
import cufflinks as cf
cf.set_config_file(offline=True)

## Plotting Line Chart

In [ ]:
df['Close'].iplot(kind='line', title='Line Chart')


In [ ]:
# Next, we'll plot the time series data in ohlc format.

df[-30:].iplot(kind='ohlc', title='Bar Chart')


In [ ]:
# Next, we'll plot an interactive candlestick chart.

df[-30:].iplot(kind='candle', title='Candle Chart')


In [ ]:
# Plotting Selected Stocks

# Use secondary axis
df4[['GS', 'HD']].iplot(secondary_y='HD')

In [ ]:
# Plotting subplots
# Use subplots
df4[['GS', 'HD']].iplot(subplots=True)

In [ ]:
# Normalize plot
df4.normalize().iplot()

## Visualising Return Series
We'll now plot historical daily log normal return series using just one line of code.

In [ ]:
# Calculating Log Normal Returns
# Use numpy log function to derive log normal returns
daily_returns = np.log(df4).diff().dropna()

# Display the last five rows of the data frame to check the output
daily_returns.head()

In [ ]:
# Plot Mean Annual Returns
(daily_returns.mean()*252*100).iplot(kind='bar')

In [ ]:
# Plot Mean Annualized Volatility
(daily_returns.std()*np.sqrt(252)*100).iplot(kind='bar')

In [ ]:
# To calculate 5 days rolling returns, simply sum daily returns for 5 days as log returns are additive
rolling_return = daily_returns.rolling(5).sum().dropna()

# Display the last five rows of the data frame to check the output
rolling_return.head()

In [ ]:
# Plot Rolling Returns
rolling_return.iplot(title='5-Days Rolling Returns')

Time Series Statistics
Statistics is a branch of mathematics that deals with collecting, interpreting, organization and interpretation of data. The two main categories of statistics are descriptive statistics and inferential statistics.

Descriptive statistics help us to understand the data in a meaningful way and is an important part of data analysis. While inferential statistics allows us to infer trends and derive conclusion from it.

In [ ]:
# Analysing the daily returns data
daily_returns.describe().T

## Log Normal Distribution
A normal distribution is the most common and widely used distribution in statistics. It is popularly referred as a “bell curve” or “Gaussian curve”. Financial time series though random in short term, follows a log normal distribution on a longer time frame.

Now that we have derived the daily log returns, we will plot this return distribution and check whether the stock returns follows log normality.

In [ ]:
# Plot log normal distribution of returns
daily_returns.iplot(kind='histogram', title = 'Histogram of Daily Returns', subplots=True)

In [ ]:
# Plot correlation of returns
daily_returns.corr().iplot(kind='heatmap', title="Correlation Matrix", colorscale="Blues")